In [243]:
input_list = [3, 5, 10, 251, 10, 251, 3, 5, 10, 251, 3]

# Building and initializing the dictionary.
dictionary_size = 256
dictionary = {tuple([i]): i for i in range(dictionary_size)}

string = []  # String is initially empty
compressed_data = []  # Variable to store the compressed data

# LZW Compression algorithm
for symbol in input_list:
    string_plus_symbol = string + [symbol]  # Get input symbol.
    if tuple(string_plus_symbol) in dictionary:
        string = string_plus_symbol
    else:
        compressed_data.append(dictionary[tuple(string)])
        dictionary[tuple(string_plus_symbol)] = dictionary_size
        dictionary_size += 1
        string = [symbol]

if tuple(string) in dictionary:
    compressed_data.append(dictionary[tuple(string)])

# Output the compressed data and the final state of the dictionary
print(compressed_data)
print(dictionary)



[3, 5, 10, 251, 258, 256, 260]
{(0,): 0, (1,): 1, (2,): 2, (3,): 3, (4,): 4, (5,): 5, (6,): 6, (7,): 7, (8,): 8, (9,): 9, (10,): 10, (11,): 11, (12,): 12, (13,): 13, (14,): 14, (15,): 15, (16,): 16, (17,): 17, (18,): 18, (19,): 19, (20,): 20, (21,): 21, (22,): 22, (23,): 23, (24,): 24, (25,): 25, (26,): 26, (27,): 27, (28,): 28, (29,): 29, (30,): 30, (31,): 31, (32,): 32, (33,): 33, (34,): 34, (35,): 35, (36,): 36, (37,): 37, (38,): 38, (39,): 39, (40,): 40, (41,): 41, (42,): 42, (43,): 43, (44,): 44, (45,): 45, (46,): 46, (47,): 47, (48,): 48, (49,): 49, (50,): 50, (51,): 51, (52,): 52, (53,): 53, (54,): 54, (55,): 55, (56,): 56, (57,): 57, (58,): 58, (59,): 59, (60,): 60, (61,): 61, (62,): 62, (63,): 63, (64,): 64, (65,): 65, (66,): 66, (67,): 67, (68,): 68, (69,): 69, (70,): 70, (71,): 71, (72,): 72, (73,): 73, (74,): 74, (75,): 75, (76,): 76, (77,): 77, (78,): 78, (79,): 79, (80,): 80, (81,): 81, (82,): 82, (83,): 83, (84,): 84, (85,): 85, (86,): 86, (87,): 87, (88,): 88, (89,): 89

In [244]:
compressed_data

[3, 5, 10, 251, 258, 256, 260]

In [245]:
# Read the compressed data from the file
compressed_data = [3, 5, 10, 251, 258, 256, 260]

# Building and initializing the dictionary.
dictionary_size = 256
dictionary = {i: [i] for i in range(dictionary_size)}

# Initialize variables for decoding
previous_code = compressed_data[0]
decoded_string = dictionary[previous_code]
decoded_data = list(decoded_string)

# Decoding the compressed data
for code in compressed_data[1:]:
    if code in dictionary:
        current_string = dictionary[code]
    elif code == len(dictionary):
        current_string = decoded_string + [decoded_string[0]]
    else:
        raise ValueError("Bad compressed k")

    decoded_data += current_string
    dictionary[len(dictionary)] = decoded_string + [current_string[0]]
    decoded_string = current_string

# Output the decoded data
print(decoded_data)

[3, 5, 10, 251, 10, 251, 3, 5, 10, 251, 3]


In [246]:
def lzw_encode(data):
    # Building and initializing the dictionary.
    dictionary_size = 256
    dictionary = {tuple([i]): i for i in range(dictionary_size)}

    string = []  # String is initially empty
    compressed_data = []  # Variable to store the compressed data

    # LZW Compression algorithm
    for symbol in data:
        string_plus_symbol = string + [symbol]  # Get input symbol.
        if tuple(string_plus_symbol) in dictionary:
            string = string_plus_symbol
        else:
            compressed_data.append(dictionary[tuple(string)])
            dictionary[tuple(string_plus_symbol)] = dictionary_size
            dictionary_size += 1
            string = [symbol]

    if tuple(string) in dictionary:
        compressed_data.append(dictionary[tuple(string)])

    return compressed_data

In [247]:
def lzw_decode(compressed_data):
    # Building and initializing the dictionary.
    dictionary_size = 256
    dictionary = {i: [i] for i in range(dictionary_size)}

    # Initialize variables for decoding
    previous_code = compressed_data[0]
    decoded_string = dictionary[previous_code]
    decoded_data = list(decoded_string)

    # Decoding the compressed data
    for code in compressed_data[1:]:
        if code in dictionary:
            current_string = dictionary[code]
        elif code == len(dictionary):
            current_string = decoded_string + [decoded_string[0]]
        else:
            raise ValueError("Bad compressed k")

        decoded_data += current_string
        dictionary[len(dictionary)] = decoded_string + [current_string[0]]
        decoded_string = current_string

    return decoded_data

In [248]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [249]:
def lzw_encode_one_channel(data):
    encoded_image = []
    for line in data:
        encoded_line = lzw_encode(line)
        encoded_image.append(encoded_line)
    return encoded_image

In [250]:
def lzw_decode_one_channel(data):
    decoded_image = []
    for line in data:
        decoded_line = lzw_decode(line)
        decoded_image.append(decoded_line)
    return decoded_image

In [251]:
def lzw_encode_rgb_image(data):
    r, g, b = cv2.split(data)
    r_encoded = lzw_encode_one_channel(r)
    g_encoded = lzw_encode_one_channel(g)
    b_encoded = lzw_encode_one_channel(b)
    return [r_encoded, g_encoded, b_encoded]

In [252]:
def lzw_decode_rgb_image(data):
    r_d, g_d, b_d = data

    r_decoded = lzw_decode_one_channel(r_d)
    g_decoded = lzw_decode_one_channel(g_d)
    b_decoded = lzw_decode_one_channel(b_d)

    r_array = np.array(r_decoded)
    g_array = np.array(g_decoded)
    b_array = np.array(b_decoded)

    return cv2.merge([r_array, g_array, b_array])


In [253]:
wm_image = cv2.imread('images\WM.png', cv2.IMREAD_GRAYSCALE)

dog =  cv2.imread('images\doggy.jpg')
dog = cv2.cvtColor(dog, cv2.COLOR_BGR2RGB)

cat = cv2.imread('images\catty.jpg')
cat = cv2.cvtColor(cat, cv2.COLOR_BGR2RGB)


In [254]:
wm_encoded = lzw_encode_one_channel(wm_image)


In [255]:
wm_decoded = lzw_decode_one_channel(wm_encoded)

In [256]:
np.array_equal(wm_decoded, wm_image)

True

In [257]:
cat_encoded = lzw_encode_rgb_image(cat)

In [258]:
cat_decoded = lzw_decode_rgb_image(cat_encoded)

In [259]:
np.array_equal(cat_decoded, cat)

True

In [260]:
import sys

# Оценка размера в байтах для cat и cat_encoded
size_cat_bytes = sys.getsizeof(cat)
size_cat_encoded_bytes = sum([sys.getsizeof(ch) for ch in cat_encoded])

print("Размер cat в байтах:", size_cat_bytes)
print("Размер cat_encoded в байтах:", size_cat_encoded_bytes)


Размер cat в байтах: 172644
Размер cat_encoded в байтах: 6600


In [261]:
import pickle

# Преобразование списка cat_encoded в байты
cat_encoded_bytes = pickle.dumps(cat_encoded)

# Открытие файла для записи в двоичном режиме
output_file_path = "cat_encoded.lzw"
with open(output_file_path, "wb") as output_file:
    output_file.write(cat_encoded_bytes)

In [262]:
import pickle

# Открытие файла для чтения в двоичном режиме
with open("cat_encoded.lzw", "rb") as file:
    binary_data = file.read()

# Декодирование байтовой строки
encoded_file = pickle.loads(binary_data)


In [263]:
file_decoded = lzw_decode_rgb_image(encoded_file)

In [264]:
np.array_equal(cat_decoded, file_decoded)

True

In [265]:
np.array_equal(encoded_file, cat_encoded)

False

In [272]:
len(cat_encoded[0][0]),len(encoded_file[0][0])

(192, 192)

In [273]:
encoded_file == cat_encoded

True